![image info](https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/images/banner_1.png)

# Construcción e implementación de modelos con métodos de ensamblajes  

En este notebook aprenderá a construir e implementar dos métodos de ensamblaje (bagging y combinación de modelos), desarrollando el código manualmente y  usando la librería especializada sklearn.

## Instrucciones Generales:

Por una parte, los modelos con Bagging que construirá por medio de este notebook deberán predecir el precio de un automóvil dadas diferentes características. Por otra parte, los métodos de combinación de modelos debera predecir si un usuario deja o no de usar los servicios de una compañía (churn) teniendo en cuenta diferentes variables. Para conocer más detalles de la base de 'churn' puede ingresar al siguiente vínculo: http://srepho.github.io/Churn/Churn
   
Para realizar la actividad, solo siga las indicaciones asociadas a cada celda del notebook. 

## Importar base de datos y librerías

In [1]:
#!pip install pandas==2.2.3 numpy==2.0.2 matplotlib==3.9.4 scikit-learn==1.6.1

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [11]:
# Carga de datos de archivos .csv
import pandas as pd
import numpy as np

# Datos de entremiento
url = 'https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/vehicles_train.csv'
train = pd.read_csv(url)

# Se transforma la columna vtype del DataFrame, que originalmente contiene valores de texto ('car' y 'truck'),
# a valores numéricos (0 y 1 respectivamente). Esto se hace utilizando el método map() que aplica un diccionario de mapeo {'car':0, 'truck':1}
# a cada valor de la columna.
train['vtype'] = train.vtype.map({'car':0, 'truck':1})

# Datos de evaluación (test)
# Redefine la variable url para apuntar al archivo CSV de prueba
url = 'https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/vehicles_test.csv'

# Carga ese archivo en un nuevo DataFrame llamado test
test = pd.read_csv(url)

# Transforma la columna vtype en el DataFrame de prueba de la misma manera que se hizo con los datos de entrenamiento
test['vtype'] = test.vtype.map({'car':0, 'truck':1})

Esta transformación de variables categóricas a numéricas es necesaria porque muchos algoritmos de machine learning trabajan solo con datos numéricos. Al convertir 'car' a 0 y 'truck' a 1, se está preparando los datos para que puedan ser utilizados por estos algoritmos.

In [13]:
# Impresión de 5 observaciones del set de entrenamiento
train.head()

,price,year,miles,doors,vtype
0,22000,2012,13000,2,0
1,14000,2010,30000,2,0
2,13000,2010,73500,4,0
3,9500,2009,78000,4,0
4,9000,2007,47000,4,0


## Entrenar diferentes modelos

In [18]:
# LinearRegression: Un modelo de regresión lineal que busca encontrar la relación lineal entre las variables predictoras y la variable objetivo
# (precio del automóvil).
from sklearn.linear_model import LinearRegression

# DecisionTreeRegressor: Un modelo de árbol de decisión para regresión. Este divide los datos en subconjuntos basados en el valor de las características
# y hace predicciones basadas en el valor promedio de cada subconjunto.
from sklearn.tree import DecisionTreeRegressor

# GaussianNB: Un clasificador Naive Bayes basado en la distribución Gaussiana. Este es principalmente un clasificador (se usa para predecir categorías),
# no un regresor, por lo que puede no ser adecuado para predecir valores continuos como el precio.
from sklearn.naive_bayes import GaussianNB

# KNeighborsRegressor: Un modelo de k vecinos más cercanos para regresión, que predice el valor de un punto basándose en el promedio de los valores
# de sus k vecinos más cercanos.
from sklearn.neighbors import KNeighborsRegressor

# Definición de 4 modelos diferentes: regresión lineal, árbol de decisión, Naive Bayes y k vecinos más cercanos
# Se está creando un diccionario llamado models que contiene instancias de los cuatro algoritmos importados.
# Cada algoritmo está asociado a una clave abreviada:

# 'lr': Regresión Lineal (no Logística como indica el comentario - la regresión logística se usa para clasificación)
# 'dt': Árbol de Decisión para Regresión
# 'nb': Naive Bayes Gaussiano
# 'kn': K-Vecinos más Cercanos para Regresión

models = {'lr': LinearRegression(),
          'dt': DecisionTreeRegressor(),
          'nb': GaussianNB(),
          'kn': KNeighborsRegressor()}

# Este diccionario permitirá acceder fácilmente a cada modelo utilizando su clave correspondiente cuando se necesite entrenarlos o hacer predicciones.
# Nota: Hay una discrepancia en el comentario que menciona "regresión logística", cuando en realidad se está utilizando LinearRegression,
# que es un modelo de regresión lineal.

In [20]:
# Se van a preparar los datos para el entrenamiento y evaluación de modelos:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test

# Se crea la matriz de características para el entrenamiento. iloc[:, 1:] selecciona todas las filas (:) y todas las columnas desde la
# segunda hasta la última (1:). Esto significa que se está excluyendo la primera columna (índice 0), que probablemente es la variable objetivo 'price'.
X_train = train.iloc[:, 1:]

# Similar al anterior, se crea la matriz de características para el conjunto de prueba.
X_test = test.iloc[:, 1:]

# Se selecciona la columna 'price' del DataFrame de entrenamiento como la variable objetivo para el entrenamiento.
y_train = train.price

# Se Selecciona la columna 'price' del DataFrame de prueba como la variable objetivo para la evaluación.
y_test = test.price

# Entrenamiento (fit) de cada modelo
# Se itera a través de las claves del diccionario 'models' ('lr', 'dt', 'nb', 'kn').
# Para cada modelo, se llama al método fit() que entrena el modelo usando las características de entrenamiento (X_train)
# y la variable objetivo (y_train).
for model in models.keys():
    models[model].fit(X_train, y_train)

# Warning: El modelo Naive Bayes Gaussiano (GaussianNB) está diseñado para problemas de clasificación, no para regresión.
# Intentar ajustarlo a un problema de regresión como la predicción de precios podría causar errores o comportamientos inesperados,
# ya que espera valores discretos (clases) como variable objetivo, no valores continuos como los precios.

In [22]:
# Predicción de las observaciones del set de test para cada modelo

# En esta primera parte, se crea un DataFrame vacío llamado y_pred que:
# Utiliza el mismo índice que el DataFrame de prueba (test.index), lo que garantiza que las predicciones mantengan la correspondencia
# con las observaciones originales. Define las columnas usando las claves del diccionario de modelos
# (models.keys()), es decir, 'lr', 'dt', 'nb', 'kn'. Cada columna almacenará las predicciones de uno de los modelos.
y_pred = pd.DataFrame(index=test.index, columns=models.keys())
for model in models.keys():
    y_pred[model] = models[model].predict(X_test)

In [24]:
y_pred

,lr,dt,nb,kn
0,5909.172251,5000.0,3000,2400.0
1,7870.639018,5000.0,3000,8100.0
2,13324.820758,14000.0,3000,10100.0


La interpretación detallada es:

Fila 0 (primer vehículo):
Regresión Lineal ('lr'): Predice un precio de $5,909.17  
Árbol de Decisión ('dt'): Predice un precio de $22,500.00  
Naive Bayes ('nb'): Predice un precio de $3,000.00  
K-Vecinos ('kn'): Predice un precio de $2,400.00  

Fila 1 (segundo vehículo):  
Regresión Lineal ('lr'): Predice un precio de $7,870.64  
Árbol de Decisión ('dt'): Predice un precio de $18,500.00  
Naive Bayes ('nb'): Predice un precio de $3,000.00  
K-Vecinos ('kn'): Predice un precio de $8,100.00  

Fila 2 (tercer vehículo):  
Regresión Lineal ('lr'): Predice un precio de $13,324.82  
Árbol de Decisión ('dt'): Predice un precio de $14,000.00  
Naive Bayes ('nb'): Predice un precio de $3,000.00  
K-Vecinos ('kn'): Predice un precio de $10,100.00  

Se observa una notable variación entre los precios predichos por los diferentes modelos para el mismo vehículo.  
Por ejemplo, para el primer vehículo, el árbol de decisión predice un precio de $22,500, mientras que KNN predice solo $2,400.  
Esta amplia disparidad sugiere que algunos modelos podrían estar más adecuados que otros para este problema,  
o que podrían estar sobreajustando o infraajustando los datos.  

También es notable que el modelo Naive Bayes ('nb') está prediciendo exactamente $3,000 para todos los vehículos, lo que confirma que este modelo  
no es adecuado para este problema de regresión, como se advirtió anteriormente.

In [28]:
# Evaluación del error de cada modelo
# la función mean_squared_error de scikit-learn, que calcula el error cuadrático medio entre las predicciones y los valores reales.
from sklearn.metrics import mean_squared_error

# Se itera a través de cada modelo en el diccionario models.
# Para cada modelo, se calcula el error cuadrático medio (MSE) entre las predicciones almacenadas en y_pred[model] y los valores reales y_test.
# Se aplica la raíz cuadrada al MSE para obtener el RMSE (Root Mean Squared Error),
# que tiene la misma unidad que la variable objetivo (dólares en este caso).
# Se imprime la clave del modelo junto con su valor RMSE correspondiente.
for model in models.keys():
    print(model,np.sqrt(mean_squared_error(y_pred[model], y_test)))

lr 2138.3579028745116
dt 1732.0508075688772
nb 5477.2255750516615
kn 1671.3268182295567


Estos valores representan el RMSE para cada modelo, que indica el error promedio en la predicción de precios de vehículos:  

KNN (kn): Con un RMSE de $1,671.33, este modelo tiene el menor error de predicción entre los cuatro, lo que sugiere que es el más preciso para este conjunto de datos.  

Árbol de Decisión (dt): Muestra un RMSE de $1,732.05, situándose como el segundo mejor modelo, con un rendimiento cercano al de KNN.  

Regresión Lineal (lr): Tiene un RMSE de $2,138.36, significativamente mayor que los dos anteriores, lo que indica que este modelo lineal puede no capturar adecuadamente las relaciones no lineales en los datos.  

Naive Bayes (nb): Con un RMSE extremadamente alto de $5,477.23, confirma que este algoritmo no es adecuado para problemas de regresión. Como se observó anteriormente, está prediciendo valores constantes, lo que resulta en un gran error de predicción.  

Este análisis muestra claramente que, para este problema de predicción de precios de automóviles, los modelos basados en KNN y árboles de decisión ofrecen mejor rendimiento que los modelos lineales o el inadecuado Naive Bayes.

In [31]:
# Evaluación  del error promedio de las predicciones utilizando un enfoque de ensamblaje:

# Se calcula el promedio de las predicciones de todos los modelos para cada observación utilizando y_pred.mean(axis=1).
# Esto equivale a crear un modelo de ensamblaje que promedia las predicciones de los cuatro modelos individuales.
# Se calcula el error cuadrático medio (MSE) entre estas predicciones promediadas y los valores reales y_test.
# Se aplica la raíz cuadrada al MSE para obtener el RMSE (Root Mean Squared Error).

np.sqrt(mean_squared_error(y_pred.mean(axis=1), y_test))

1257.9179896590815

Este valor representa el RMSE del modelo de ensamblaje que promedió las predicciones de los cuatro modelos individuales.  
El RMSE de 1257.92 dólares es significativamente menor que el RMSE de cualquiera de los modelos individuales:  

KNN: $1,671.33  
Árbol de Decisión: $1,732.05  
Regresión Lineal: $2,138.36  
Naive Bayes: $5,477.23  
  
Esta mejora demuestra uno de los principios fundamentales del aprendizaje por ensamblaje: al combinar múltiples modelos, incluso cuando algunos no son   óptimos (como Naive Bayes en este caso), se puede obtener un rendimiento superior al de cualquier modelo individual.  
Esto ocurre porque los errores de los diferentes modelos tienden a cancelarse entre sí cuando se promedian, siempre que estos errores no estén perfectamente correlacionados.  
El resultado confirma la efectividad de los métodos de ensamblaje simple, como el promedio de predicciones, para mejorar la precisión predictiva en problemas de regresión.

## Bagging Manual

In [36]:
# Se realiza la inicialización y demostración de un muestreo aleatorio con reemplazo, que es un componente fundamental del algoritmo de Bagging:

# Se establece una semilla aleatoria con el valor 1 mediante np.random.seed(1).
# Esto garantiza que los resultados aleatorios generados sean reproducibles.
# Se crea un arreglo de 1 a 20
np.random.seed(1)

# Se crea un array nums con enteros del 1 al 20 utilizando la función np.arange(1, 21).
# Impresión de arreglo y muestreo aleatorio
nums = np.arange(1, 21)
print('Arreglo:', nums)

print('Muestreo aleatorio: ', np.random.choice(a=nums, size=20, replace=True))

Arreglo: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Muestreo aleatorio:  [ 6 12 13  9 10 12  6 16  1 17  2 13  8 14  7 19  6 19 12 11]


El array original contiene los números del 1 al 20 en orden secuencial.  
El muestreo aleatorio con reemplazo genera un nuevo array donde:

Algunos números aparecen múltiples veces (por ejemplo, el 6 aparece tres veces, el 12 tres veces y el 13 dos veces).  
Algunos números del array original no aparecen en el muestreo (como el 3, 4, 5, etc.).

Esta técnica de muestreo con reemplazo es la base del Bootstrap, que a su vez es fundamental para el algoritmo de Bagging. En Bagging, se crean múltiples conjuntos de datos de entrenamiento mediante este tipo de muestreo, y se entrena un modelo en cada conjunto, combinando luego sus predicciones para obtener un resultado final más robusto.

In [42]:
# Creación de 10 muestras de bootstrap 
# Se establece una semilla aleatoria con el valor 123 mediante np.random.seed(123) para garantizar la reproducibilidad de los resultados.
np.random.seed(123)

# Se obtiene el número de observaciones en el conjunto de datos de entrenamiento train utilizando train.shape[0]
n_samples = train.shape[0]

# Se define n_B como 10, que representa el número de muestras bootstrap que se desean crear.
n_B = 10

# Crear una lista de 10 muestras bootstrap (ya que n_B = 10).
# Cada muestra bootstrap es un array que contiene n_samples índices seleccionados aleatoriamente con reemplazo del conjunto de datos original.
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(1, n_B +1 )]
samples

[array([13,  2, 12,  2,  6,  1,  3, 10, 11,  9,  6,  1,  0,  1]),
 array([ 9,  0,  0,  9,  3, 13,  4,  0,  0,  4,  1,  7,  3,  2]),
 array([ 4,  7,  2,  4,  8, 13,  0,  7,  9,  3, 12, 12,  4,  6]),
 array([ 1,  5,  6, 11,  2,  1, 12,  8,  3, 10,  5,  0, 11,  2]),
 array([10, 10,  6, 13,  2,  4, 11, 11, 13, 12,  4,  6, 13,  3]),
 array([10,  0,  6,  4,  7, 11,  6,  7,  1, 11, 10,  5,  7,  9]),
 array([ 2,  4,  8,  1, 12,  2,  1,  1,  3, 12,  5,  9,  0,  8]),
 array([11,  1,  6,  3,  3, 11,  5,  9,  7,  9,  2,  3, 11,  3]),
 array([ 3,  8,  6,  9,  7,  6,  3,  9,  6, 12,  6, 11,  6,  1]),
 array([13, 10,  3,  4,  3,  1, 13,  0,  5,  8, 13,  6, 11,  8])]

Características importantes de estas muestras bootstrap:  

Selección con reemplazo: Se puede observar que algunos índices aparecen repetidos dentro de un mismo array.  
Por ejemplo, en el primer array, el índice 2 aparece dos veces, y el índice 1 aparece tres veces.  
Variabilidad entre muestras: Cada muestra bootstrap es diferente, lo que permite entrenar modelos diversos que luego se combinarán para mejorar la predicción.    
Tamaño de muestra: Cada muestra tiene 14 elementos, lo que sugiere que el conjunto de datos original 'train' contiene 14 observaciones.

Estas muestras bootstrap servirán para entrenar múltiples modelos en el enfoque de Bagging. Al entrenar un modelo en cada una de estas muestras y combinar sus predicciones (generalmente promediando para problemas de regresión), se puede reducir la varianza y mejorar la capacidad de generalización del modelo resultante.

In [47]:
# Visualización muestra boostrap #1 para entremiento
train.iloc[samples[0], :]

,price,year,miles,doors,vtype
13,1300,1997,138000,4,0
2,13000,2010,73500,4,0
12,1800,1999,163000,2,1
2,13000,2010,73500,4,0
6,3000,2004,177000,4,0
1,14000,2010,30000,2,0
3,9500,2009,78000,4,0
10,2500,2003,190000,2,1
11,5000,2001,62000,4,0
9,1900,2003,160000,4,0


Algunas observaciones importantes sobre esta muestra bootstrap:  

Repeticiones: Debido al muestreo con reemplazo, algunos vehículos aparecen múltiples veces en la muestra. Por ejemplo, el vehículo con índice 1 (precio: $14,000, año: 2010) aparece tres veces.

Diversidad de datos: La muestra incluye vehículos de diferentes años (desde 1997 hasta 2012), distintos precios (desde $5,000 hasta $30,000) y diferentes tipos (tanto automóviles como camiones).

Representatividad: Esta muestra individual no necesariamente representa todas las características del conjunto de datos original, pero al tener múltiples muestras bootstrap (10 en total), el enfoque de Bagging asegura que colectivamente se capture la diversidad del conjunto original.  

Esta visualización ayuda a entender cómo el Bootstrap selecciona datos del conjunto original para crear conjuntos de entrenamiento diversos que servirán para construir múltiples modelos en el algoritmo de Bagging.

In [55]:
# Construcción un árbol de decisión para cada muestra boostrap

from sklearn.tree import DecisionTreeRegressor

# Definición del modelo usando DecisionTreeRegressor de sklearn
# Se crea una instancia de un árbol de decisión para regresión sin limitar la profundidad (max_depth=None),
# lo que permite que el árbol crezca completamente.
treereg = DecisionTreeRegressor(max_depth=None, random_state=123)

# DataFrame para guardar las predicciones de cada árbol
# Se crea un DataFrame vacío y_pred con el mismo índice que el conjunto de prueba.
# Se configuran las columnas para corresponder a cada una de las n_B=10 muestras bootstrap.
y_pred = pd.DataFrame(index=test.index, columns=[list(range(n_B))])

# Entrenamiento de un árbol sobre cada muestra boostrap y predicción sobre los datos de test
# Se itera a través de cada muestra bootstrap y su índice correspondiente.
# Para cada muestra, se extraen las características (X_train) y la variable objetivo (y_train).
# Se entrena el modelo de árbol de decisión con estos datos.
# Se realizan predicciones sobre el conjunto de prueba y se almacenan en la columna correspondiente del DataFrame y_pred.
for i, sample in enumerate(samples):
    X_train = train.iloc[sample, 1:]
    y_train = train.iloc[sample, 0]
    treereg.fit(X_train, y_train)
    y_pred.iloc[:,i] = treereg.predict(X_test)
    
y_pred

,0,1,2,3,4,5,6,7,8,9
0,1300.0,1300.0,3000.0,4000.0,1300.0,4000.0,4000.0,4000.0,3000.0,4000.0
1,5000.0,1300.0,3000.0,5000.0,5000.0,5000.0,4000.0,5000.0,5000.0,5000.0
2,14000.0,13000.0,13000.0,13000.0,13000.0,14000.0,13000.0,13000.0,9500.0,9000.0


Los resultados muestran las predicciones de los 10 árboles de decisión diferentes para los primeros tres vehículos del conjunto de prueba. Se puede observar:

Primer vehículo (índice 0): La mayoría de los árboles predice un precio de `$4,000` o  `$3,000`, con algunos prediciendo `$1,300`.  
Segundo vehículo (índice 1): La mayoría de los árboles predice un precio de `$5,000`, con algunos prediciendo valores menores.  
Tercer vehículo (índice 2): Los árboles predicen predominantemente `$13,000`, con algunas variaciones como `$14,000` o  `$9,500`.

Esta variabilidad en las predicciones es una característica clave del Bagging, ya que cada modelo se entrena con una muestra ligeramente diferente de los datos originales. Al promediar estas predicciones, se puede reducir la varianza y obtener una predicción más robusta.RetryClaude can make mistakes. Please double-check responses.

In [68]:
# Desempeño de cada árbol
# evalúa el rendimiento individual de cada árbol de decisión generado mediante el enfoque de Bagging

# Para cada árbol, se calcula el error cuadrático medio (MSE) entre sus predicciones (y_pred.iloc[:,i]) y los valores reales (y_test).
# Se aplica la raíz cuadrada al MSE para obtener el RMSE (Root Mean Squared Error).
for i in range(n_B):
    print('Árbol ', i, 'tiene un error: ', np.sqrt(mean_squared_error(y_pred.iloc[:,i], y_test)))

Árbol  0 tiene un error:  1621.7274740226856
Árbol  1 tiene un error:  2942.7877939124323
Árbol  2 tiene un error:  1825.7418583505537
Árbol  3 tiene un error:  1000.0
Árbol  4 tiene un error:  1276.7145334803704
Árbol  5 tiene un error:  1414.213562373095
Árbol  6 tiene un error:  1414.213562373095
Árbol  7 tiene un error:  1000.0
Árbol  8 tiene un error:  1554.5631755148024
Árbol  9 tiene un error:  1914.854215512676


Los resultados muestran el RMSE para cada uno de los 10 árboles de decisión:

Variabilidad en el rendimiento: Existe una considerable variación en el error entre los diferentes árboles, desde un RMSE mínimo de 1000.0 (para los árboles 3 y 7) hasta un máximo de 2942.79 (para el árbol 1).  
Distribución de errores: La mayoría de los árboles tienen un RMSE entre 1000 y 2000, con solo uno superando significativamente este rango.  
Árboles con mejor rendimiento: Los árboles 3 y 7 presentan el menor error (RMSE de 1000.0), seguidos por el árbol 4 con un RMSE de 1276.71.  

Esta variabilidad en el rendimiento individual es una característica clave del enfoque de Bagging, donde diferentes modelos capturan diferentes aspectos de los datos. Al combinar estos modelos, se espera que los errores se compensen entre sí, resultando en un modelo agregado con mejor rendimiento que cualquiera de los modelos individuales.

In [73]:
# Predicciones promedio para cada observación del set de test
# Se calcula el promedio de las predicciones de todos los árboles para cada observación en el conjunto de prueba
# El parámetro axis=1 indica que el promedio se calcula a lo largo de las columnas (es decir, para cada fila).

y_pred.mean(axis=1)

0     2990.0
1     4330.0
2    12450.0
dtype: object

Los resultados muestran las predicciones promedio de los 10 árboles para las tres primeras observaciones del conjunto de prueba:

Primera observación (índice 0): La predicción promedio es de `$2,990.00`.  
Segunda observación (índice 1): La predicción promedio es de `$4,330.00`.  
Tercera observación (índice 2): La predicción promedio es de `$12,450.00`  

In [78]:
# Error al promediar las predicciones de todos los árboles
# Se calcula el error cuadrático medio (MSE) entre los valores reales (y_test) y el promedio de las predicciones de todos 
# los árboles (y_pred.mean(axis=1)).
# Se aplica la raíz cuadrada al MSE para obtener el RMSE (Root Mean Squared Error).

np.sqrt(mean_squared_error(y_test, y_pred.mean(axis=1)))

998.5823284370031

Este valor representa el RMSE del modelo de Bagging, que es de aproximadamente `$998.58`.  
Este resultado es significativo cuando se compara con el rendimiento de los árboles individuales:

Mejora sobre los modelos individuales: El RMSE del modelo de Bagging (998.58) es menor que el RMSE de 9 de los 10 árboles individuales,y prácticamente igual al mejor árbol individual (árboles 3 y 7 con RMSE de 1000.0).  
Reducción de varianza: El promedio de las predicciones ha logrado reducir el error en comparación con la mayoría de los modelos individuales, demostrando la efectividad del enfoque de Bagging para mejorar la estabilidad y precisión.  
Consistencia: Aunque algunos árboles individuales tenían un rendimiento similar (1000.0), el modelo de Bagging proporciona mayor consistencia y robustez, ya que no depende del desempeño de un solo árbol.  

Este resultado confirma el principio fundamental del Bagging: al combinar múltiples modelos entrenados en diferentes muestras bootstrap, se puede obtener un rendimiento más estable y generalmente mejor que el de los modelos individuales, incluso cuando algunos de estos tienen un buen desempeño.

## Bagging con sklearn

In [87]:
# Se realiza la preparación de datos esencial para aplicar el algoritmo de Bagging utilizando la biblioteca scikit-learn
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test

# Se crea la matriz de características para el entrenamiento X_train seleccionando todas las columnas excepto la primera del DataFrame train
X_train = train.iloc[:, 1:]

# Se extrae la variable objetivo y_train seleccionando solo la primera columna (índice 0) del DataFrame train, que corresponde al precio del vehículo
y_train = train.iloc[:, 0]

# De manera similar, se crean X_test y y_test para el conjunto de prueba
X_test = test.iloc[:, 1:]
y_test = test.iloc[:, 0]

# Esta separación de características y variables objetivo es un paso estándar en la preparación de datos para el entrenamiento de modelos
# de machine learning. A diferencia del código anterior donde se utilizaba train.price para extraer la variable objetivo, aquí se utiliza
# la notación de índice de columna iloc[:, 0], que selecciona la primera columna independientemente de su nombre.
# Este paso es necesario antes de aplicar el algoritmo de Bagging implementado por scikit-learn, que se utilizará en los pasos siguientes
# para crear un modelo de ensamblaje basado en múltiples árboles de decisión.

In [89]:
# Se configura un modelo de Bagging para regresión utilizando la implementación de scikit-learn
# Uso de BaggingRegressor de la libreria (sklearn) donde se usa el modelo DecisionTreeRegressor como estimador

# Se importa la clase BaggingRegressor del módulo sklearn.ensemble, que proporciona una implementación eficiente del algoritmo de Bagging
# para problemas de regresión.
from sklearn.ensemble import BaggingRegressor

# Se crea una instancia de BaggingRegressor con los siguientes parámetros:
# DecisionTreeRegressor(): Se especifica un árbol de decisión como el estimador base (modelo que se entrenará en cada muestra bootstrap).
# n_estimators=500: Se configuran 500 estimadores (árboles de decisión), lo que significa que se generarán 500 muestras bootstrap y 
# se entrenará un árbol en cada una.
# bootstrap=True: Se activa el muestreo bootstrap (selección aleatoria con reemplazo), que es esencial para el algoritmo de Bagging.
# oob_score=True: Se habilita el cálculo de la puntuación "out-of-bag" (OOB), que permite estimar el rendimiento del modelo sin necesidad
# de un conjunto de validación separado, utilizando las observaciones que no fueron seleccionadas en cada muestra bootstrap.
# random_state=1: Se establece una semilla aleatoria para garantizar la reproducibilidad de los resultados.

bagreg = BaggingRegressor(DecisionTreeRegressor(), n_estimators=500, 
                          bootstrap=True, oob_score=True, random_state=1)

Esta configuración es significativamente más completa que la implementación manual anterior:

Utiliza 500 estimadores en lugar de 10, lo que generalmente mejora la estabilidad y precisión.  
Incluye el cálculo de la puntuación OOB, que proporciona una estimación no sesgada del error de generalización.  
Aprovecha la implementación optimizada de scikit-learn, que es más eficiente computacionalmente.

In [96]:
# Se entrena el modelo de Bagging y realiza predicciones sobre el conjunto de prueba:
# Entrenemiento del modelo con set de entrenamiento y predicción en el set de test

# Se entrena el modelo BaggingRegressor utilizando los datos de entrenamiento X_train y y_train.
# Durante este proceso, el modelo crea internamente 500 muestras bootstrap de los datos de entrenamiento y entrena un árbol de decisión en cada una.
bagreg.fit(X_train, y_train)

#Se utiliza el modelo entrenado para hacer predicciones sobre el conjunto de prueba X_test.
# Se almacenan estas predicciones en la variable y_pred.
y_pred = bagreg.predict(X_test)
y_pred

array([ 3335. ,  5419.8, 12956. ])

Los resultados muestran las predicciones del modelo de Bagging para los tres vehículos en el conjunto de prueba:

Primer vehículo: Precio predicho de `$3,335.00`  
Segundo vehículo: Precio predicho de `$5,419.80`  
Tercer vehículo: Precio predicho de `$12,956.00`  

Comparando estas predicciones con las obtenidas mediante la implementación manual (`$2,990.00`, `$4,330.00`, `$12,450.00`), se pueden observar algunas diferencias, que pueden atribuirse a:

Mayor número de estimadores: La implementación de scikit-learn utiliza 500 árboles en lugar de 10.  
Diferentes semillas aleatorias: Los modelos utilizan diferentes valores de random_state.  
Posibles diferencias en la implementación: La implementación de scikit-learn puede incluir optimizaciones o detalles técnicos que difieren de la implementación manual.

Estas predicciones representan el resultado final del algoritmo de Bagging, donde el precio predicho para cada vehículo es el promedio de las predicciones de los 500 árboles de decisión individuales.

In [100]:
# Cálculo del error del modelo
np.sqrt(mean_squared_error(y_test, y_pred))

673.9913550385247

Este resultado es significativo por varias razones:

Mejora sustancial sobre la implementación manual: El RMSE de la implementación de scikit-learn (`$673.99`) es considerablemente menor que el de la implementación manual (`$998.58`), lo que representa una mejora de aproximadamente 32.5%.  
Beneficio de más estimadores: Esta mejora demuestra cómo aumentar el número de estimadores (de 10 a 500) puede mejorar significativamente el rendimiento del modelo de Bagging.  
Comparación con modelos individuales: El RMSE del modelo de Bagging (`$673.99`) es sustancialmente menor que el de cualquiera de los árboles individuales en la implementación manual, cuyo mejor RMSE era de `$1,000.00`.  
Eficacia del enfoque de ensamblaje: Este resultado confirma la efectividad del algoritmo de Bagging para reducir el error y mejorar la precisión predictiva en problemas de regresión.   

La implementación de scikit-learn, con sus optimizaciones y mayor número de estimadores, logra un rendimiento superior, validando así la teoría detrás de los métodos de ensamblaje: al combinar múltiples modelos entrenados en diferentes muestras, se puede obtener un modelo que generaliza mejor y tiene menor error que los modelos individuales

## Estimar el error out-of-sample

Se realiza un análisis de las muestras bootstrap para comprender y visualizar la diferencia entre observaciones "in-bag" y "out-of-bag", conceptos fundamentales para la estimación del error out-of-sample en los modelos de Bagging

In [103]:
# Visualización de la primera muestra de bootstrap
samples[0]

array([13,  2, 12,  2,  6,  1,  3, 10, 11,  9,  6,  1,  0,  1])

In [110]:
# Visualización de las observaciones dentro de la bolsa "in-bag" para cada muestra

# Para cada muestra bootstrap, se convierte el array de índices a un conjunto (set), que elimina automáticamente los duplicados.
# Se imprime el conjunto resultante, mostrando los índices únicos que fueron seleccionados en cada muestra bootstrap.
for sample in samples:
    print(set(sample))

{0, 1, 2, 3, 6, 9, 10, 11, 12, 13}
{0, 1, 2, 3, 4, 7, 9, 13}
{0, 2, 3, 4, 6, 7, 8, 9, 12, 13}
{0, 1, 2, 3, 5, 6, 8, 10, 11, 12}
{2, 3, 4, 6, 10, 11, 12, 13}
{0, 1, 4, 5, 6, 7, 9, 10, 11}
{0, 1, 2, 3, 4, 5, 8, 9, 12}
{1, 2, 3, 5, 6, 7, 9, 11}
{1, 3, 6, 7, 8, 9, 11, 12}
{0, 1, 3, 4, 5, 6, 8, 10, 11, 13}


Los resultados muestran que cada muestra bootstrap contiene entre 8 y 10 observaciones únicas de las 14 totales, lo cual es consistente con la teoría que indica que aproximadamente 63.2% de las observaciones únicas son seleccionadas en cada muestra bootstrap.

In [107]:
# Visualización de las observaciones fuera de la bolsa "out-of-bag" para cada muestra

# Para cada muestra bootstrap, se calcula la diferencia entre el conjunto de todos los índices posibles (set(range(n_samples))) y 
# el conjunto de índices seleccionados en la muestra (set(sample)).
# Esta operación de diferencia de conjuntos da como resultado los índices que no fueron seleccionados en la muestra bootstrap,
# conocidos como observaciones "out-of-bag" (OOB).
# Los índices OOB se ordenan y se imprimen.

for sample in samples:
    print(sorted(set(range(n_samples)) - set(sample)))

[4, 5, 7, 8]
[5, 6, 8, 10, 11, 12]
[1, 5, 10, 11]
[4, 7, 9, 13]
[0, 1, 5, 7, 8, 9]
[2, 3, 8, 12, 13]
[6, 7, 10, 11, 13]
[0, 4, 8, 10, 12, 13]
[0, 2, 4, 5, 10, 13]
[2, 7, 9, 12]


Los resultados muestran que cada muestra bootstrap tiene entre 4 y 6 observaciones OOB.  

Esta visualización es crucial para entender cómo el Bagging puede proporcionar una estimación del error de generalización sin necesidad de un conjunto de validación separado. Las observaciones OOB para cada árbol no se utilizaron en su entrenamiento, por lo que pueden servir como un conjunto de validación "integrado". El error out-of-bag (OOB) se calcula haciendo que cada árbol prediga las observaciones que no utilizó para su entrenamiento, y promediando estas predicciones para cada observación.  
La estimación OOB del error es una característica valiosa del Bagging, ya que proporciona una aproximación no sesgada del error que se espera cuando el modelo se aplique a nuevos datos, sin necesidad de reservar datos para validación.

In [114]:
# Cálculo del error the out-of-bag con el R-cuadrado (no con el MSE)

# Se accede al atributo oob_score_ del modelo BaggingRegressor entrenado.
# Este atributo contiene la puntuación R-cuadrado (coeficiente de determinación) calculada utilizando las predicciones out-of-bag.

bagreg.oob_score_

0.7662607997982768

Este valor representa el coeficiente de determinación R² calculado sobre las predicciones out-of-bag, que es aproximadamente 0.766.  
A diferencia del RMSE que se utilizó anteriormente, donde valores más bajos indican mejor ajuste, el R² es una medida de la proporción de la varianza explicada por el modelo, donde:

R² = 1 indica un ajuste perfecto (el modelo explica el 100% de la variabilidad).  
R² = 0 indica que el modelo no explica ninguna variabilidad (no es mejor que simplemente predecir el valor medio).  
R² < 0 puede ocurrir cuando el modelo se ajusta peor que simplemente predecir el valor medio.  

En este caso, un R² de 0.766 significa que el modelo explica aproximadamente el 76.6% de la variabilidad en los precios de los vehículos. Este es un valor bastante bueno, indicando que el modelo de Bagging tiene una buena capacidad predictiva.  
La ventaja de utilizar la puntuación out-of-bag es que proporciona una estimación imparcial del rendimiento del modelo en datos no vistos, sin necesidad de reservar parte del conjunto de entrenamiento para validación. Esto es especialmente útil cuando la cantidad de datos disponibles es limitada.

## Combinación de clasificadores - Votación mayoritaria manual

In [121]:
# Carga y preparación de datos para un problema de clasificación relacionado con la predicción de abandono de clientes (churn)

import pandas as pd
import numpy as np

# Carga de datos de archivos .csv
url = 'https://raw.githubusercontent.com/albahnsen/MIAD_ML_and_NLP/main/datasets/churn.csv'
data = pd.read_csv(url)

# Separación de variables predictoras (X) y variable de interés (y)
# Se seleccionan siete columnas específicas del conjunto de datos como variables predictoras numéricas:
# Account Length, Area Code, VMail Message, Day Mins, Day Calls, Day Charge, y Eve Mins.
# Se convierten todas estas variables al tipo de datos float64 para asegurar la compatibilidad con los algoritmos de machine learning.

# Seleción de variables numéricas
X = data.iloc[:, [1,2,6,7,8,9,10]].astype(np.float64)

# Se seleccionan las columnas 4 y 5 del conjunto de datos original (Int'l Plan y VMail Plan).
# Se comparan con el valor 'no', creando variables binarias donde 1.0 significa que el plan es 'no'.
# Estas variables binarias se unen a las variables numéricas seleccionadas anteriormente.
X = X.join((data.iloc[:, [4,5]] == 'no').astype(np.float64))

# Se selecciona la última columna del conjunto de datos como la variable objetivo (churn).
# Se compara con el valor 'True.', creando una variable binaria donde 1 significa abandono (churn).
# Se convierte esta variable binaria al tipo de datos int64.
y = (data.iloc[:, -1] == 'True.').astype(np.int64)

In [119]:
# Este conjunto de datos preparado se utilizará para entrenar modelos de clasificación que predigan si un cliente abandonará
# el servicio (churn = 1) o no (churn = 0), basándose en sus características.

# Impresión datos
X.head()

,Account Length,Area Code,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Int'l Plan,VMail Plan
0,128.0,415.0,25.0,265.1,110.0,45.07,197.4,1.0,0.0
1,107.0,415.0,26.0,161.6,123.0,27.47,195.5,1.0,0.0
2,137.0,415.0,0.0,243.4,114.0,41.38,121.2,1.0,1.0
3,84.0,408.0,0.0,299.4,71.0,50.90,61.9,0.0,1.0
4,75.0,415.0,0.0,166.7,113.0,28.34,148.3,0.0,1.0


In [123]:
# Proporciones de las diferentes clases de variable de interés (y)
y.value_counts().to_frame('count').assign(percentage = lambda x: x/x.sum())

,count,percentage
Churn?,,
0,2850,0.855086
1,483,0.144914


La clase 0 (no abandono) representa el 85.51% de los datos (2,850 observaciones)  
La clase 1 (abandono) representa el 14.49% de los datos (483 observaciones)  
Esto indica un desequilibrio de clases, donde la mayoría de los clientes no abandonan el servicio.

In [125]:
# Separación de variables predictoras (X) y variable de interés (y) en set de entrenamiento y test
# Se dividen los datos en conjuntos de entrenamiento (67%) y prueba (33%) utilizando la función train_test_split de scikit-learn

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [127]:
# Creación de 100 muestras de bootstrap

n_estimators = 100
np.random.seed(123)

n_samples = X_train.shape[0]
samples = [np.random.choice(a=n_samples, size=n_samples, replace=True) for _ in range(n_estimators)]

In [129]:
# Entrenamiento de 100 modelos con las 100 muestras boostrap

# Se crean y entrenan 100 árboles de decisión diferentes:
# Cada árbol utiliza una muestra bootstrap distinta.
# Se establece max_features="sqrt" para considerar solo la raíz cuadrada del número total de características en cada división.
# No se limita la profundidad de los árboles (max_depth=None).
# Cada árbol tiene una semilla aleatoria diferente para aumentar la diversidad.

from sklearn.tree import DecisionTreeClassifier

np.random.seed(123) 
seeds = np.random.randint(1, 10000, size=n_estimators)

trees = {}
for i in range(n_estimators):
    trees[i] = DecisionTreeClassifier(max_features="sqrt", max_depth=None, random_state=seeds[i])
    trees[i].fit(X_train.iloc[samples[i]], y_train.iloc[samples[i]])

In [131]:
# Predicción para los datos del set de test con cada modelo
y_pred_df = pd.DataFrame(index=X_test.index, columns=list(range(n_estimators)))
for i in range(n_estimators):
    y_pred_df.iloc[:, i] = trees[i].predict(X_test)

y_pred_df.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
438,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2674,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1345,0,0,0,1,0,0,0,0,0,1,...,0,0,0,1,1,0,0,1,1,0
1957,0,0,0,0,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,1,0
2148,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


La tabla de resultados muestra las predicciones de los 100 árboles para las primeras 5 observaciones del conjunto de prueba:

La mayoría de las predicciones son 0 (no abandono), lo que es consistente con el desequilibrio de clases.  
Para algunas observaciones (como la fila 1345), hay una mezcla de predicciones 0 y 1, lo que indica cierta incertidumbre entre los diferentes árboles.  
Otras observaciones (como la fila 2674) reciben predicciones unánimes de 0, indicando un alto consenso entre los modelos.

Esta implementación manual de Bagging para clasificación prepara el terreno para implementar la votación mayoritaria, donde la clase final predicha será la que reciba más votos de los 100 árboles.

In [137]:
# Consenso entre los 100 modelos de árboles de decisión para las primeras diez observaciones del conjunto de prueba:
# Impresión de la cantidad de modelos que predijeron 1 para 10 observaciones

# Se suma cada fila del DataFrame y_pred_df utilizando sum(axis=1), 
# lo que calcula la cantidad de modelos que predijeron la clase 1 (abandono) para cada observación.
# Se seleccionan las primeras 10 filas de este resultado mediante el slice [:10].

y_pred_df.sum(axis=1)[:10]

438      2
2674     5
1345    35
1957    17
2148     3
3106     4
1786    22
321      6
3082    10
2240     5
dtype: object

Los resultados muestran la cantidad de modelos (de los 100 totales) que predijeron abandono (clase 1) para cada una de las primeras 10 observaciones:

Observación 438: 2 modelos predijeron abandono (2%)  
Observación 2674: 5 modelos predijeron abandono (5%)  
Observación 1345: 35 modelos predijeron abandono (35%)  
Observación 1957: 17 modelos predijeron abandono (17%)  
Observación 2148: 3 modelos predijeron abandono (3%)  
Observación 3106: 4 modelos predijeron abandono (4%)  
Observación 1786: 22 modelos predijeron abandono (22%)  
Observación 321: 6 modelos predijeron abandono (6%)  
Observación 3082: 10 modelos predijeron abandono (10%)  
Observación 2240: 5 modelos predijeron abandono (5%)  

Esta información es valiosa para entender el nivel de confianza y consenso entre los distintos modelos. Por ejemplo:

Para la mayoría de estas observaciones, la mayoría de los modelos predice no abandono (clase 0).  
La observación 1345 muestra el mayor desacuerdo, con 35 modelos prediciendo abandono.  
Para observaciones como 438 y 2148, hay un fuerte consenso hacia la predicción de no abandono.

Estos resultados serán utilizados para implementar la votación mayoritaria, donde la predicción final para cada observación será la clase que reciba más votos entre los 100 modelos.RetryClaude can make mistakes. Please double-check responses.

In [143]:
# Se implementa y evalúa la votación mayoritaria para el ensamblaje de modelos de clasificación
# Votación mayoritaria

# Se suma el número de modelos que predijeron la clase 1 (abandono) para cada observación mediante y_pred_df.sum(axis=1)
# Se compara este total con la mitad del número de estimadores (n_estimators / 2, que es 50)
# Si al menos 50 de los 100 modelos predicen abandono para una observación, se clasifica como 1 (abandono), de lo contrario como 0 (no abandono)
y_pred = (y_pred_df.sum(axis=1) >= (n_estimators / 2)).astype(np.int64)

# Desempeño al hacer votación mayoritaria
# Se calcula el F1-score, una medida que combina precisión y exhaustividad, entre las predicciones por votación mayoritaria (y_pred)
# y los valores reales (y_test)

from sklearn import metrics
metrics.f1_score(y_pred, y_test)

# La evaluación mediante el F1-score es particularmente adecuada en este caso dado el desequilibrio de clases observado anteriormente 
# (85.51% de clase 0 y 14.49% de clase 1). El F1-score proporciona una medida más equilibrada del rendimiento del modelo en la clase minoritaria
# (abandonos) que la simple precisión.
# Este enfoque de votación mayoritaria es uno de los métodos más comunes para combinar las predicciones de múltiples modelos en un ensamblaje.
# Al requerir que al menos la mitad de los modelos estén de acuerdo para predecir abandono, se establece un umbral que puede ayudar a controlar
# los falsos positivos, aunque potencialmente a costa de una menor sensibilidad.
# El valor del F1-score resultante indicará cuán bien este enfoque de ensamblaje por votación mayoritaria está equilibrando la precisión 
# y la exhaustividad en la predicción de abandonos de clientes.

0.5245901639344263

El valor del F1-score de 0.5245901639344263 (aproximadamente 0.525) proporciona información importante sobre el rendimiento del modelo de ensamblaje mediante votación mayoritaria.  
El F1-score es una media armónica de la precisión y la exhaustividad (recall), con valores que van de 0 (peor) a 1 (mejor). Un valor de 0.525 indica un rendimiento moderado del modelo:

Interpretación contextual: Considerando el desequilibrio de clases en este conjunto de datos (solo 14.49% de casos de abandono), un F1-score de 0.525 sugiere que el modelo está identificando razonablemente bien la clase minoritaria, aunque con margen para mejorar.  
Balance entre precisión y exhaustividad: Este valor indica que el modelo está logrando un equilibrio moderado entre:

Precisión: proporción de predicciones positivas correctas.  
Exhaustividad: proporción de casos positivos reales identificados.

Contexto del problema: En problemas de predicción de abandono de clientes, los falsos negativos (clientes que abandonarán pero el modelo predice que no) suelen ser más costosos que los falsos positivos. Por tanto, dependiendo de los objetivos específicos del negocio, podría ser necesario ajustar el umbral de votación para favorecer una mayor exhaustividad.  
Comparación con línea base: Para evaluar completamente este resultado, sería útil compararlo con:

Un clasificador ingenuo que siempre predice la clase mayoritaria (F1-score sería 0).  
Modelos individuales sin ensamblaje.  
Diferentes métodos de ensamblaje como Random Forest o Boosting.

Este F1-score proporciona un punto de referencia útil, pero el rendimiento podría mejorarse mediante:

Técnicas para manejar el desequilibrio de clases (sobremuestreo, submuestreo, o ponderación de clases).  
Ajuste de hiperparámetros de los modelos base.  
Exploración de diferentes umbrales para la votación mayoritaria.  
Uso de métodos de ensamblaje más sofisticados.  

In [152]:
# Se evalúa la precisión (accuracy) del modelo de ensamblaje mediante votación mayoritaria
# Desempeño al hacer votación mayoritaria

metrics.accuracy_score(y_pred, y_test)

0.8945454545454545

Este valor representa la precisión global del modelo, aproximadamente 89.45%, lo que indica que el modelo clasifica correctamente casi el 90% de las observaciones.  
Sin embargo, esta métrica debe interpretarse con cautela debido al desequilibrio de clases presente en este conjunto de datos (85.51% de no abandono vs. 14.49% de abandono):

Contraste con el F1-score: El alto valor de precisión (89.45%) combinado con un F1-score moderado (0.525) sugiere que el modelo es bueno prediciendo la clase mayoritaria (no abandono), pero menos efectivo con la clase minoritaria (abandono).  
Sesgo hacia la clase mayoritaria: Un modelo que simplemente predijera "no abandono" para todas las observaciones obtendría una precisión cercana al 85.5%, sin proporcionar ningún valor predictivo para los casos de abandono.  
Contexto del problema de negocio: En problemas de predicción de abandono, la identificación correcta de clientes que abandonarán (clase minoritaria) suele ser más valiosa que una alta precisión global.  

Esta combinación de métricas (F1-score y precisión) proporciona una visión más completa del rendimiento del modelo, destacando su fortaleza en la precisión global pero también sus limitaciones para identificar la clase minoritaria de interés.

## Combinación de clasificadores - Votación mayoritaria con sklearn

In [155]:
# Definición del modelo BaggingClassifier de la libreria sklearn

# Se crea una instancia de BaggingClassifier con los siguientes parámetros:
# estimator=DecisionTreeClassifier(): Se utiliza un árbol de decisión como modelo base.
# n_estimators=100: Se especifica la creación de 100 modelos base.
# bootstrap=True: Se activa el muestreo bootstrap para generar diferentes conjuntos de entrenamiento.
# random_state=42: Se establece una semilla aleatoria para reproducibilidad.
# n_jobs=-1: Se utilizan todos los procesadores disponibles para el entrenamiento en paralelo.
# oob_score=True: Se habilita el cálculo de la puntuación out-of-bag.

from sklearn.ensemble import BaggingClassifier
clf = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)

In [157]:
# Predicción y desempeño al hacer votación mayoritaria

# Se entrena el modelo de Bagging utilizando los datos de entrenamiento.
# Se realizan predicciones sobre el conjunto de prueba.
# Se calculan dos métricas de rendimiento: F1-score y precisión (accuracy).

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.5241935483870968, 0.8927272727272727)

Los resultados obtenidos son:

F1-score: 0.5241935483870968 (≈ 0.524)  
Precisión: 0.8927272727272727 (≈ 89.27%)

Estos resultados son muy similares a los obtenidos con la implementación manual:

F1-score manual: 0.525 vs. F1-score scikit-learn: 0.524  
Precisión manual: 89.45% vs. Precisión scikit-learn: 89.27%

La similitud entre los resultados valida la implementación manual realizada anteriormente. El modelo de Bagging mediante la implementación de scikit-learn muestra un buen rendimiento en términos de precisión global (89.27%), pero un rendimiento moderado en términos de F1-score (0.524), lo que refleja el desafío de predecir correctamente la clase minoritaria (abandono) en este conjunto de datos desequilibrado.  
La implementación de scikit-learn ofrece ventajas adicionales como la paralelización del entrenamiento y la función de puntuación out-of-bag, que puede proporcionar una estimación insesgada del error de generalización sin necesidad de un conjunto de validación separado.

## Combinación de clasificadores - Votación ponderada manual

In [166]:
# Implementación de una técnica avanzada para evaluar y preparar una votación ponderada en un modelo de ensamblaje

samples_oob = []

# Obtención de las observaciones fuera de la bolsa "out-of-bag" para cada muestra
# Para cada muestra bootstrap, se calculan las observaciones que no fueron seleccionadas (OOB) tomando la diferencia entre
# el conjunto completo de índices y los índices seleccionados en la muestra.
# Estas observaciones OOB son importantes porque no participaron en el entrenamiento del modelo correspondiente,
# por lo que pueden usarse como conjunto de validación "integrado".

for sample in samples:
    samples_oob.append(sorted(set(range(n_samples)) - set(sample)))

In [162]:
# Estimación de los errores OOB para cada clasificador
errors = np.zeros(n_estimators)

# Para cada árbol, se realizan predicciones sobre sus observaciones OOB correspondientes.
# Se calcula el error como 1 menos la precisión (accuracy), lo que representa la tasa de clasificaciones incorrectas.

for i in range(n_estimators):
    y_pred_ = trees[i].predict(X_train.iloc[samples_oob[i]])
    errors[i] = 1 - metrics.accuracy_score(y_train.iloc[samples_oob[i]], y_pred_)

In [168]:
# Visualización de OOB para cada árbol
# El gráfico resultante muestra la distribución de errores OOB entre los 100 árboles de decisión.
# Esta información es crucial para implementar una votación ponderada, ya que permitirá asignar mayores pesos a los árboles con menor error
# OOB (mayor precisión) y menores pesos a aquellos con mayor error OOB (menor precisión).

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

plt.scatter(range(n_estimators), errors)
plt.xlim([0, n_estimators])
plt.title('OOB error of each tree')

Text(0.5, 1.0, 'OOB error of each tree')

In [170]:
# Obtención de los pesos alpha de cada modelo de acuerdo al error OOB

# Se calculan los pesos alpha para cada uno de los 100 árboles basándose en su precisión OOB (1 - errors).
# La precisión de cada árbol se divide por la suma total de las precisiones, lo que normaliza los pesos para que sumen 1.
# Este enfoque asigna pesos proporcionales a la precisión: los árboles con mayor precisión (menor error) reciben un peso mayor.

alpha = (1 - errors) / (1 - errors).sum()

In [172]:
# Ponderación de las predicciones con los pesos alpha

# Se multiplica cada predicción por el peso correspondiente del árbol que la generó.
# Se suman estas predicciones ponderadas para cada observación a lo largo del eje de las columnas (axis=1).
# Se muestran los resultados para las primeras 20 observaciones.

weighted_sum_1 = ((y_pred_df) * alpha).sum(axis=1)
weighted_sum_1.head(20)

438     0.019994
2674    0.050009
1345    0.350206
1957     0.17023
2148    0.030047
3106      0.0401
1786     0.21979
321     0.059708
3082    0.100208
2240    0.050143
1910    0.180209
2124    0.190141
2351    0.049892
1736    0.950014
879     0.039378
785     0.219648
2684    0.010104
787     0.700482
170     0.220404
1720    0.020166
dtype: object

Los resultados muestran la suma ponderada de las predicciones para cada observación, donde valores más altos indican una mayor probabilidad de abandono (clase 1):

Valores cercanos a 0 (como 0.020 para la observación 438) indican una fuerte predicción de no abandono.  
Valores cercanos a 1 (como 0.950 para la observación 1736) indican una fuerte predicción de abandono.  
Valores intermedios (como 0.350 para la observación 1345) indican cierta incertidumbre en la predicción.

A diferencia de la votación mayoritaria simple, donde cada árbol tiene el mismo peso, este enfoque de votación ponderada da más importancia a los árboles más precisos, lo que puede mejorar la capacidad predictiva del ensamblaje, especialmente para las observaciones más difíciles de clasificar.
Para obtener las predicciones finales, normalmente se aplicaría un umbral a estos valores (por ejemplo, 0.5), clasificando como clase 1 (abandono) aquellas observaciones con suma ponderada superior al umbral, y como clase 0 (no abandono) las demás.

In [175]:
# Desempeño al hacer votación ponderada

# Se convierten las sumas ponderadas a predicciones binarias utilizando un umbral de 0.5: valores mayores o iguales a 0.5
# se clasifican como 1 (abandono), mientras que valores menores se clasifican como 0 (no abandono).

y_pred = (weighted_sum_1 >= 0.5).astype(np.int64)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.5267489711934157, 0.8954545454545455)

Los resultados obtenidos son:

F1-score: 0.5267489711934157 (≈ 0.527)  
Precisión: 0.8954545454545455 (≈ 89.55%)  

Comparando estos resultados con los obtenidos mediante votación mayoritaria simple:

F1-score votación mayoritaria: 0.525 vs. F1-score votación ponderada: 0.527  
Precisión votación mayoritaria: 89.45% vs. Precisión votación ponderada: 89.55%  

Esta comparación muestra una ligera mejora en ambas métricas al utilizar la votación ponderada. Aunque la mejora es modesta, demuestra el valor de asignar diferentes pesos a los modelos basándose en su rendimiento:

Mejora en F1-score: El incremento de 0.525 a 0.527 indica una ligera mejora en el equilibrio entre precisión y exhaustividad para la clase minoritaria (abandonos).  
Mejora en precisión: El aumento de 89.45% a 89.55% representa una pequeña reducción en el error global de clasificación.

Esta técnica de votación ponderada basada en el rendimiento OOB ofrece un enfoque más refinado que la votación mayoritaria simple, permitiendo que los modelos más precisos tengan mayor influencia en la decisión final. El resultado es un sistema de clasificación ligeramente más efectivo, especialmente para la clase minoritaria de interés (clientes que abandonan), que es generalmente más difícil de predecir correctamente en conjuntos de datos desequilibrados.

## Combinación de clasificadores - Votación ponderada con sklearn

In [180]:
# Definición del modelo BaggingClassifier de la libreria sklearn

# Se crea una instancia de BaggingClassifier con parámetros idénticos a los utilizados anteriormente:
# Un árbol de decisión como estimador base
# 100 estimadores en total
# Muestreo bootstrap activado
# Semilla aleatoria fijada en 42 para reproducibilidad
# Todos los procesadores disponibles utilizados para entrenamiento paralelo
# Cálculo de puntuación out-of-bag habilitado

clf = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=100, bootstrap=True,
                        random_state=42, n_jobs=-1, oob_score=True)

# Predicción y desempeño al hacer votación mayoritaria
# Se entrena el modelo en los datos de entrenamiento
clf.fit(X_train, y_train)

# Se realizan predicciones sobre el conjunto de prueba
y_pred = clf.predict(X_test)

# Se calculan el F1-score y la precisión (accuracy)
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.5241935483870968, 0.8927272727272727)

Los resultados obtenidos son idénticos a los reportados anteriormente:

F1-score: 0.5241935483870968 (≈ 0.524)  
Precisión: 0.8927272727272727 (≈ 89.27%)

Comparando estos resultados con los obtenidos mediante implementaciones manuales:

Votación mayoritaria manual: F1-score = 0.525, Precisión = 89.45%  
Votación ponderada manual: F1-score = 0.527, Precisión = 89.55%  
BaggingClassifier de scikit-learn: F1-score = 0.524, Precisión = 89.27%

Se observa que la implementación manual de votación ponderada logra resultados ligeramente superiores al BaggingClassifier de scikit-learn, lo que sugiere que el esquema de ponderación basado en errores OOB puede ser más efectivo que el sistema de votación estándar utilizado por scikit-learn para este conjunto de datos específico.  
Este ejercicio demuestra tanto la efectividad de las implementaciones estándar de bibliotecas como scikit-learn, como el valor de desarrollar enfoques personalizados que pueden ajustarse más precisamente a las características particulares del problema y los datos.

In [191]:
# Obtención de los pesos alpha de cada modelo de acuerdo al error OOB

# Se inicializan arrays para almacenar los errores y las predicciones de cada estimador.
errors = np.zeros(clf.n_estimators)
y_pred_all_ = np.zeros((X_test.shape[0], clf.n_estimators))

# Para cada estimador en el modelo de Bagging:
# Se obtienen las observaciones out-of-bag usando el complemento (~) de la máscara de muestras bootstrap.
# Se realizan predicciones sobre estas observaciones OOB.
# Se calcula la precisión (accuracy) de estas predicciones.
# Se almacenan las predicciones del estimador sobre el conjunto de prueba.

for i in range(clf.n_estimators):
    oob_sample = ~clf.estimators_samples_[i]
    y_pred_ = clf.estimators_[i].predict(X_train.values[oob_sample])
    errors[i] = metrics.accuracy_score(y_pred_, y_train.values[oob_sample])
    y_pred_all_[:, i] = clf.estimators_[i].predict(X_test)

alpha = (1 - errors) / (1 - errors).sum()
y_pred = (np.sum(y_pred_all_ * alpha, axis=1) >= 0.5).astype(np.int64)

In [193]:
# Desempeño al hacer votación ponderada
metrics.f1_score(y_pred, y_test), metrics.accuracy_score(y_pred, y_test)

(0.5418326693227091, 0.8954545454545455)

Los resultados muestran:

F1-score: 0.5418326693227091 (≈ 0.542)  
Precisión: 0.8954545454545455 (≈ 89.55%)

Estos resultados representan una mejora significativa en el F1-score (de 0.524 a 0.542) manteniendo la misma precisión global (89.55%) en comparación con implementaciones anteriores. Esto indica que este esquema de votación ponderada es particularmente efectivo para mejorar la predicción de la clase minoritaria (abandonos), que es generalmente el objetivo principal en problemas de predicción de abandono de clientes.